# Feature engineering - Adjusting revenue 
To predict whether or not sequels or films inspired by novels are more successful. 

## Tasks this notebook achieves
- Problem: budget has a large number of missing values
    - [x] given mean value to rows without budget, this is better than removing as there are a large amount of empty rows
-  Problem: revenue data is not good enough
    - [x] I removed zero revenue rows, resulting in 900 rows lost. Not great, but I can’t predict revenue without revenue.
    - [x] I remove year to only take into account the seasonality of a movie release

In [1]:
import os
import sys

In [2]:
sys.path.insert(0, os.path.abspath('../utils/')) #point this to the where util is relatively to your working directory
from util import *

data_dir = get_path_to_data_dir()

In [3]:
import pandas as pd
import json 
import numpy as np
from datetime import datetime

In [4]:
movie_details_join = pd.read_pickle(data_dir + 'pre-processed/movie_encoded_all.pkl')

movie_details_join.set_index('id').head()

,budget,popularity,revenue,runtime,vote_average,vote_count,genres,keywords,original_language,original_title,...,Yasiin Bey,Yul Vazquez,Zac Efron,Zach Galifianakis,Zachary Woodlee,Zak Penn,Zeljko Ivanek,Zhang Ziyi,Zoe Saldana,Zooey Deschanel
id,,,,,,,,,,,,,,,,,,,,,
64682,105000000,61.196071,351040419,143.0,7.3,3769,"{'id': [18, 10749], 'name': ['Drama', 'Romance']}","{'id': [818, 1326, 1523, 3929, 209714], 'name'...",en,The Great Gatsby,...,0,0,0,0,0,0,0,0,0,0
9543,150000000,62.169881,335154643,116.0,6.2,2317,"{'id': [12, 14, 28, 10749], 'name': ['Adventur...","{'id': [1241, 1965, 12653, 12654, 12655, 41645...",en,Prince of Persia: The Sands of Time,...,0,0,0,0,0,0,0,0,0,0
5174,140000000,22.571780,258022233,91.0,6.1,783,"{'id': [28, 35, 80, 53], 'name': ['Action', 'C...","{'id': [1704], 'name': ['ambassador']}",en,Rush Hour 3,...,0,0,0,0,0,0,0,0,0,0
1735,145000000,60.034162,401128639,112.0,5.2,1387,"{'id': [12, 28, 14], 'name': ['Adventure', 'Ac...","{'id': [2280, 3996, 160153, 207372, 13154, 131...",en,The Mummy: Tomb of the Dragon Emperor,...,0,0,0,0,0,0,0,0,0,0
79698,27000000,2.418535,0,109.0,4.8,34,"{'id': [28, 12, 878, 10749], 'name': ['Action'...","{'id': [6917, 9840, 3667, 9935, 4414, 196288, ...",en,The Lovers,...,0,0,0,0,0,0,0,0,0,0


We want to select independent variables for training during machine learning modelling. I ignored some non-useful variables, such as film title and homepage. Obviously these can’t be used to predict the success of a movie.

Some variables were discarded for other reasons: production_country, because I felt that the information therein would be stored in production_company. Original_language, because I felt that that column would mostly be covered by spoken_languages, with a few exceptions. Popularity, because obviously that was measured after the film was released.

In [5]:
movie_neat_filter = movie_details_join.drop(columns=["original_title", "original_language", "popularity"], axis=1)

In [6]:
# meaning out 0 budgets - there are a lot, so this is better than removing the rows
movie_neat_filter['budget']=movie_neat_filter['budget'].replace(0,movie_neat_filter['budget'].median())

The variables used for model prediction were:
User vote (akin to IMDb rating, referred to as ‘rating’ throughout)
User-reported box office revenue (referred to as ‘revenue’ throughout)

In [7]:
# Removing zero REVENUES from the data - revenue is super important as this will 
# be one of the variables we want to predict

def remove_zero_revenue(y_revenue, y_rating, X):
    y_revenue_removed = []
    y_rating_removed = []
    X_removed = []

    for l in range(0,len(y_revenue)):
        if y_revenue[l] !=0:
            y_revenue_removed.append(y_revenue[l])
            y_rating_removed.append(y_rating[l])
            X_removed.append(X[l])        
    y_revenue = np.array(y_revenue_removed)
    y_rating = np.array(y_rating_removed)
    X = np.array(X_removed)
    return y_revenue, y_rating, X

In [8]:
X = movie_neat_filter.iloc[:, :].values
y_revenue = movie_neat_filter.iloc[:, 2].values
y_rating = movie_neat_filter.iloc[:, 4].values

In [9]:
#storing removed rows in numpy arrays
y_revenue_nonull, y_rating_nonull, X_nonull = remove_zero_revenue(y_revenue, y_rating, X)

In [10]:
movie_filter_rev = movie_neat_filter[movie_neat_filter.revenue != 0]

In [11]:
movie_filter_rev

,budget,id,revenue,runtime,vote_average,vote_count,genres,keywords,overview,production_companies,...,Yasiin Bey,Yul Vazquez,Zac Efron,Zach Galifianakis,Zachary Woodlee,Zak Penn,Zeljko Ivanek,Zhang Ziyi,Zoe Saldana,Zooey Deschanel
0,105000000,64682,351040419,143.0,7.3,3769,"{'id': [18, 10749], 'name': ['Drama', 'Romance']}","{'id': [818, 1326, 1523, 3929, 209714], 'name'...",An adaptation of F. Scott Fitzgerald's Long Is...,"{'name': ['Village Roadshow Pictures', 'Bazmar...",...,0,0,0,0,0,0,0,0,0,0
1,150000000,9543,335154643,116.0,6.2,2317,"{'id': [12, 14, 28, 10749], 'name': ['Adventur...","{'id': [1241, 1965, 12653, 12654, 12655, 41645...",A rogue prince reluctantly joins forces with a...,"{'name': ['Walt Disney Pictures', 'Jerry Bruck...",...,0,0,0,0,0,0,0,0,0,0
2,140000000,5174,258022233,91.0,6.1,783,"{'id': [28, 35, 80, 53], 'name': ['Action', 'C...","{'id': [1704], 'name': ['ambassador']}",After an attempted assassination on Ambassador...,"{'name': ['New Line Cinema'], 'id': [12]}",...,0,0,0,0,0,0,0,0,0,0
3,145000000,1735,401128639,112.0,5.2,1387,"{'id': [12, 28, 14], 'name': ['Adventure', 'Ac...","{'id': [2280, 3996, 160153, 207372, 13154, 131...","Archaeologist Rick O'Connell travels to China,...","{'name': ['Universal Pictures', 'China Film Co...",...,0,0,0,0,0,0,0,0,0,0
5,15000000,315011,77000000,120.0,6.5,143,"{'id': [28, 12, 18, 27, 878], 'name': ['Action...","{'id': [1299, 7671, 11100, 14796, 161791, 2355...",From the mind behind Evangelion comes a hit la...,"{'name': ['Cine Bazar', 'Toho Pictures'], 'id'...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4302,14000000,33693,76901,85.0,6.3,8,"{'id': [18, 35], 'name': ['Drama', 'Comedy']}","{'id': [171993], 'name': ['mumblecore']}","Unsure of what to do next, 23-year-old Marnie ...",{},...,0,0,0,0,0,0,0,0,0,0
4313,12000,692,6000000,93.0,6.2,110,"{'id': [27, 35, 80], 'name': ['Horror', 'Comed...","{'id': [237, 900, 1740, 2231, 2671, 2843, 2874...",Notorious Baltimore criminal and underground f...,"{'name': ['Dreamland Productions'], 'id': [407]}",...,0,0,0,0,0,0,0,0,0,0
4316,20000,36095,99000,111.0,7.4,63,"{'id': [80, 27, 9648, 53], 'name': ['Crime', '...","{'id': [233, 549, 612, 818, 3298, 4434, 5340, ...",A wave of gruesome murders is sweeping Tokyo. ...,"{'name': ['Daiei Studios'], 'id': [881]}",...,0,0,0,0,0,0,0,0,0,0
4319,7000,14337,424760,77.0,6.9,658,"{'id': [878, 18, 53], 'name': ['Science Fictio...","{'id': [1448, 2101, 3394, 4379, 5455, 6009, 10...",Friends/fledgling entrepreneurs invent a devic...,"{'name': ['Thinkfilm'], 'id': [446]}",...,0,0,0,0,0,0,0,0,0,0


We want to remove the year from our data as we will not be addressing revenue prediction using time series analysis, and just take into account the seasonality of the release.

In [12]:
# converting film date to day of year
# i am arguably losing the 'year' which might be slightly correlated with film success
# but that opens up a whole new can of worms about ratings and revenues by year
def remove_year(df):
    datetime_object = list(map(lambda x: datetime.strptime(x,'%Y-%m-%d'), df['release_date']))
    datetime_tuple_ls = []
    for l in datetime_object:
        datetime_tuple = l.timetuple().tm_yday
        datetime_tuple_ls.append(datetime_tuple)
    df['release_date'] = datetime_tuple_ls
    df = df.reset_index(drop=True)
    return df

In [13]:
movie_no_year = remove_year(movie_filter_rev)

/home/klc116/.local/share/virtualenvs/datatonic-challenge-vXlODfEJ/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [14]:
#Inspecting processed dataframe
movie_no_year

,budget,id,revenue,runtime,vote_average,vote_count,genres,keywords,overview,production_companies,...,Yasiin Bey,Yul Vazquez,Zac Efron,Zach Galifianakis,Zachary Woodlee,Zak Penn,Zeljko Ivanek,Zhang Ziyi,Zoe Saldana,Zooey Deschanel
0,105000000,64682,351040419,143.0,7.3,3769,"{'id': [18, 10749], 'name': ['Drama', 'Romance']}","{'id': [818, 1326, 1523, 3929, 209714], 'name'...",An adaptation of F. Scott Fitzgerald's Long Is...,"{'name': ['Village Roadshow Pictures', 'Bazmar...",...,0,0,0,0,0,0,0,0,0,0
1,150000000,9543,335154643,116.0,6.2,2317,"{'id': [12, 14, 28, 10749], 'name': ['Adventur...","{'id': [1241, 1965, 12653, 12654, 12655, 41645...",A rogue prince reluctantly joins forces with a...,"{'name': ['Walt Disney Pictures', 'Jerry Bruck...",...,0,0,0,0,0,0,0,0,0,0
2,140000000,5174,258022233,91.0,6.1,783,"{'id': [28, 35, 80, 53], 'name': ['Action', 'C...","{'id': [1704], 'name': ['ambassador']}",After an attempted assassination on Ambassador...,"{'name': ['New Line Cinema'], 'id': [12]}",...,0,0,0,0,0,0,0,0,0,0
3,145000000,1735,401128639,112.0,5.2,1387,"{'id': [12, 28, 14], 'name': ['Adventure', 'Ac...","{'id': [2280, 3996, 160153, 207372, 13154, 131...","Archaeologist Rick O'Connell travels to China,...","{'name': ['Universal Pictures', 'China Film Co...",...,0,0,0,0,0,0,0,0,0,0
4,15000000,315011,77000000,120.0,6.5,143,"{'id': [28, 12, 18, 27, 878], 'name': ['Action...","{'id': [1299, 7671, 11100, 14796, 161791, 2355...",From the mind behind Evangelion comes a hit la...,"{'name': ['Cine Bazar', 'Toho Pictures'], 'id'...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069,14000000,33693,76901,85.0,6.3,8,"{'id': [18, 35], 'name': ['Drama', 'Comedy']}","{'id': [171993], 'name': ['mumblecore']}","Unsure of what to do next, 23-year-old Marnie ...",{},...,0,0,0,0,0,0,0,0,0,0
3070,12000,692,6000000,93.0,6.2,110,"{'id': [27, 35, 80], 'name': ['Horror', 'Comed...","{'id': [237, 900, 1740, 2231, 2671, 2843, 2874...",Notorious Baltimore criminal and underground f...,"{'name': ['Dreamland Productions'], 'id': [407]}",...,0,0,0,0,0,0,0,0,0,0
3071,20000,36095,99000,111.0,7.4,63,"{'id': [80, 27, 9648, 53], 'name': ['Crime', '...","{'id': [233, 549, 612, 818, 3298, 4434, 5340, ...",A wave of gruesome murders is sweeping Tokyo. ...,"{'name': ['Daiei Studios'], 'id': [881]}",...,0,0,0,0,0,0,0,0,0,0
3072,7000,14337,424760,77.0,6.9,658,"{'id': [878, 18, 53], 'name': ['Science Fictio...","{'id': [1448, 2101, 3394, 4379, 5455, 6009, 10...",Friends/fledgling entrepreneurs invent a devic...,"{'name': ['Thinkfilm'], 'id': [446]}",...,0,0,0,0,0,0,0,0,0,0


### Aggregating by book
We have now preprocessed all the numeric movie data. As our research question concerns looking at movies based on sequels or on books, we chose to aggregate columns such as `"based on young adult novel", "based on novel", "based on comic book"` summarising into one column `book`. We include all the columns individually in another dataframe.

In [15]:
def based_on_book(df):
    '''
    adds all novels to the unified 'book' column
    '''
    for i in range(len(df)):
        if df.loc[i, 'based on novel'] == 1:
            df.loc[i, 'book'] = 1
        if df.loc[i, 'book'] == 1:
            continue
        if df.loc[i, 'based on comic book'] == 1:
            df.loc[i, 'book'] = 1
        if df.loc[i, 'based on young adult novel'] == 1:
            df.loc[i, 'book'] = 1
    # dropping the individual book columns to avoid duplicating information
    df = df.drop(["based on young adult novel", "based on novel", "based on comic book"], axis=1)

    return df

In [16]:
movie_with_book_summarised = based_on_book(movie_no_year)

In [17]:
movie_with_book_summarised.head()

,budget,id,revenue,runtime,vote_average,vote_count,genres,keywords,overview,production_companies,...,Yasiin Bey,Yul Vazquez,Zac Efron,Zach Galifianakis,Zachary Woodlee,Zak Penn,Zeljko Ivanek,Zhang Ziyi,Zoe Saldana,Zooey Deschanel
0,105000000,64682,351040419,143.0,7.3,3769,"{'id': [18, 10749], 'name': ['Drama', 'Romance']}","{'id': [818, 1326, 1523, 3929, 209714], 'name'...",An adaptation of F. Scott Fitzgerald's Long Is...,"{'name': ['Village Roadshow Pictures', 'Bazmar...",...,0,0,0,0,0,0,0,0,0,0
1,150000000,9543,335154643,116.0,6.2,2317,"{'id': [12, 14, 28, 10749], 'name': ['Adventur...","{'id': [1241, 1965, 12653, 12654, 12655, 41645...",A rogue prince reluctantly joins forces with a...,"{'name': ['Walt Disney Pictures', 'Jerry Bruck...",...,0,0,0,0,0,0,0,0,0,0
2,140000000,5174,258022233,91.0,6.1,783,"{'id': [28, 35, 80, 53], 'name': ['Action', 'C...","{'id': [1704], 'name': ['ambassador']}",After an attempted assassination on Ambassador...,"{'name': ['New Line Cinema'], 'id': [12]}",...,0,0,0,0,0,0,0,0,0,0
3,145000000,1735,401128639,112.0,5.2,1387,"{'id': [12, 28, 14], 'name': ['Adventure', 'Ac...","{'id': [2280, 3996, 160153, 207372, 13154, 131...","Archaeologist Rick O'Connell travels to China,...","{'name': ['Universal Pictures', 'China Film Co...",...,0,0,0,0,0,0,0,0,0,0
4,15000000,315011,77000000,120.0,6.5,143,"{'id': [28, 12, 18, 27, 878], 'name': ['Action...","{'id': [1299, 7671, 11100, 14796, 161791, 2355...",From the mind behind Evangelion comes a hit la...,"{'name': ['Cine Bazar', 'Toho Pictures'], 'id'...",...,0,0,0,0,0,0,0,0,0,0


### Creating a new dataframe containing the pre-processed data ready for modelling
We need to remove any data in data in string or object format from the table to get it model ready. This will include the two y variables (revenue and average_vote) as we will complete futher normalisation on these before splitting into train and test set. The following function does this:

In [18]:
def creating_numeric_df(df):
    """
    Takes the OHE encoded data and creates a dataframe with only columns used for modelling
    """
    #dropping duplicated "movie_id" column and setting index on movie id"
    df = df.drop(["movie_id"], axis=1)
    
    df = df.select_dtypes(include=['number'])
    
    return df

In [19]:
movie_only_numeric = creating_numeric_df(movie_with_book_summarised)
movie_only_numeric = movie_only_numeric.drop(['crew'], axis ='columns')

In [20]:
movie_only_numeric

,budget,id,revenue,runtime,vote_average,vote_count,release_date,lead_male,lead_female,lead2_male,...,Yasiin Bey,Yul Vazquez,Zac Efron,Zach Galifianakis,Zachary Woodlee,Zak Penn,Zeljko Ivanek,Zhang Ziyi,Zoe Saldana,Zooey Deschanel
0,105000000,64682,351040419,143.0,7.3,3769,130,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,150000000,9543,335154643,116.0,6.2,2317,139,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,140000000,5174,258022233,91.0,6.1,783,220,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,145000000,1735,401128639,112.0,5.2,1387,183,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,15000000,315011,77000000,120.0,6.5,143,211,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069,14000000,33693,76901,85.0,6.3,8,263,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3070,12000,692,6000000,93.0,6.2,110,72,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3071,20000,36095,99000,111.0,7.4,63,310,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3072,7000,14337,424760,77.0,6.9,658,282,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [21]:
movie_with_book_summarised.to_pickle(data_dir+"pre-processed/movie_encoded_book_summary.pkl")
movie_only_numeric.to_pickle(data_dir+"pre-processed/movie_encoded_only_numeric.pkl")